## 4. Modelado baseline en Python

En esta sección construimos los primeros modelos de clasificación para detectar correos fraudulentos (phishing vs legítimos) usando el dataset **`Cleaned_Featured_Dataset.csv`**, que ya incluye:
- Texto limpio (subject y body).
- Variables numéricas derivadas (longitud, conteo de palabras, caracteres especiales, etc.).
- La etiqueta binaria `label` (0 = legítimo, 1 = phishing/spam).

Comenzamos importando las librerías necesarias y cargando el dataset limpio.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

print("Cargando dataset limpio...")
data = pd.read_csv("Cleaned_Featured_Dataset.csv")
print("Filas:", len(data))

Cargando dataset limpio...
Filas: 10706


### 4.1 Definición de variables predictoras (X) y variable objetivo (y)

En este paso:

- Definimos la variable objetivo `y` como la columna `label`, que indica si el correo es **phishing/spam (1)** o **legítimo (0)**.
- Seleccionamos como `X` únicamente las **características numéricas** que construimos en la etapa de *feature engineering*:

  - `subject_len`, `body_len`: longitud del asunto y del cuerpo.
  - `subject_words`, `body_words`: número de palabras en asunto y cuerpo.
  - `subject_upper`, `body_upper`: proporción de mayúsculas.
  - `digits_count`: cantidad de dígitos en el cuerpo.
  - `special_chars`: cantidad de caracteres especiales.
  - `urgency_score`: presencia de palabras de urgencia típicas de phishing.
  - `email_count`, `url_count`: conteo de correos y URLs detectadas en el cuerpo.
  - `urls`: número de URLs ya extraído en el preprocesamiento.

Mostramos las formas de `X` y `y` para verificar dimensiones.

In [2]:
# ===============================
# 1. DEFINIR FEATURES Y TARGET
# ===============================

target = "label"

# Columnas numéricas que creaste en el script de limpieza/FE
feature_cols = [
    "subject_len",
    "body_len",
    "subject_words",
    "body_words",
    "subject_upper",
    "body_upper",
    "digits_count",
    "special_chars",
    "urgency_score",
    "email_count",
    "url_count",
    "urls"            # la original, por si aporta algo
]

X = data[feature_cols]
y = data[target]

print("\nShapes:")
print("X:", X.shape)
print("y:", y.shape)


Shapes:
X: (10706, 12)
y: (10706,)


### 4.2 Separación en conjuntos de entrenamiento y prueba

Dividimos el dataset en:

- `X_train`, `y_train`: 80% de los datos para **entrenar** los modelos.
- `X_test`, `y_test`: 20% de los datos para **evaluar** su desempeño.

Usamos:

- `test_size=0.2` → 20% para prueba.
- `random_state=42` → para garantizar reproducibilidad.
- `stratify=y` → para conservar el mismo balance de clases en train y test.

In [3]:
# ===============================
# 2. TRAIN / TEST SPLIT
# ===============================

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("\nTrain/Test sizes:")
print("X_train:", X_train.shape, "X_test:", X_test.shape)


Train/Test sizes:
X_train: (8564, 12) X_test: (2142, 12)


### 4.3 Escalado de variables para Regresión Logística

La **Regresión Logística** es un modelo lineal que se beneficia de tener las variables en una escala comparable.

Por eso:

- Ajustamos un `StandardScaler` con `X_train`.
- Transformamos tanto `X_train` como `X_test` a versiones escaladas (`X_train_scaled`, `X_test_scaled`).

> Nota: este escalado sólo se usará para la Regresión Logística. El Random Forest, al ser un modelo basado en árboles, no lo necesita.

In [4]:
# ===============================
# 3. ESCALADO PARA REGRESIÓN LOGÍSTICA
# ===============================

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### 4.4 Modelo 1: Regresión Logística (baseline lineal)

Entrenamos una **Regresión Logística** como modelo baseline:

- Entrenamos usando los datos **escalados** (`X_train_scaled`).
- Obtenemos predicciones de clase (`y_pred_log`) y probabilidades (`y_proba_log`).
- Calculamos las métricas de desempeño principales sobre el conjunto de prueba:
  - `Accuracy`
  - `Precision`
  - `Recall`
  - `F1`
  - `AUC` (Área bajo la curva ROC)

Finalmente, mostramos el `classification_report` para analizar el rendimiento por clase (0 = legítimo, 1 = phishing).

In [6]:
# ===============================
# 4. MODELO 1: REGRESIÓN LOGÍSTICA
# ===============================

print("\n=== MODELO 1: Regresión Logística ===")

log_clf = LogisticRegression(
    max_iter=1000,
    n_jobs=-1
)
log_clf.fit(X_train_scaled, y_train)
y_pred_log = log_clf.predict(X_test_scaled)
y_proba_log = log_clf.predict_proba(X_test_scaled)[:, 1]

print("Accuracy :", accuracy_score(y_test, y_pred_log))
print("Precision:", precision_score(y_test, y_pred_log))
print("Recall   :", recall_score(y_test, y_pred_log))
print("F1       :", f1_score(y_test, y_pred_log))
print("AUC      :", roc_auc_score(y_test, y_proba_log))

print("\nClassification report (Logistic):")
print(classification_report(y_test, y_pred_log))


=== MODELO 1: Regresión Logística ===


Accuracy : 0.8053221288515406
Precision: 0.8583860759493671
Recall   : 0.8201058201058201
F1       : 0.8388094317742559
AUC      : 0.889881010062416

Classification report (Logistic):
              precision    recall  f1-score   support

           0       0.73      0.78      0.75       819
           1       0.86      0.82      0.84      1323

    accuracy                           0.81      2142
   macro avg       0.79      0.80      0.80      2142
weighted avg       0.81      0.81      0.81      2142



### 4.5 Modelo 2: Random Forest (baseline no lineal)

Como segundo modelo baseline entrenamos un **Random Forest**, que:

- No requiere escalado de variables.
- Captura relaciones **no lineales** y posibles interacciones entre features.
- Suele ser muy competitivo para problemas tabulares.

En este bloque:

- Entrenamos el `RandomForestClassifier` con los datos originales (`X_train`).
- Calculamos predicciones de clase y probabilidades (`y_pred_rf`, `y_proba_rf`).
- Reportamos las mismas métricas (Accuracy, Precision, Recall, F1, AUC).
- Imprimimos también el `classification_report` para analizar el desempeño por clase.

In [7]:
# ===============================
# 5. MODELO 2: RANDOM FOREST
# ===============================

print("\n=== MODELO 2: RandomForest ===")

rf_clf = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    random_state=42,
    n_jobs=-1
)
rf_clf.fit(X_train, y_train)  # RandomForest no necesita escalado
y_pred_rf = rf_clf.predict(X_test)
y_proba_rf = rf_clf.predict_proba(X_test)[:, 1]

print("Accuracy :", accuracy_score(y_test, y_pred_rf))
print("Precision:", precision_score(y_test, y_pred_rf))
print("Recall   :", recall_score(y_test, y_pred_rf))
print("F1       :", f1_score(y_test, y_pred_rf))
print("AUC      :", roc_auc_score(y_test, y_proba_rf))

print("\nClassification report (RandomForest):")
print(classification_report(y_test, y_pred_rf))

print("\nModelado baseline completo.")


=== MODELO 2: RandomForest ===
Accuracy : 0.9211017740429505
Precision: 0.9397865853658537
Recall   : 0.9319727891156463
F1       : 0.9358633776091082
AUC      : 0.9731901171810469

Classification report (RandomForest):
              precision    recall  f1-score   support

           0       0.89      0.90      0.90       819
           1       0.94      0.93      0.94      1323

    accuracy                           0.92      2142
   macro avg       0.92      0.92      0.92      2142
weighted avg       0.92      0.92      0.92      2142


Modelado baseline completo.
